In [17]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import pickle
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.utils import resample

# Load in all the Data

In [18]:
with open('ei_intermediate_file_paths.json') as output_path_file:
    output_paths = json.load(output_path_file)

SI_PREDS_INPUT_PATH = output_paths.get("smoke_impact_model_predictions.csv")

EI_MODEL_OUTPUT_PATH = output_paths.get("ei_model_output")

In [19]:
# Load dataset
si_preds_df = pd.read_csv(SI_PREDS_INPUT_PATH)